# Pip installs

In [ ]:
!pip install tabulate

In [ ]:
!pip install copy

In [ ]:
!pip install requests[socks]

In [ ]:
!pip install numpy

In [ ]:
!pip install requests

In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install pandas

# Library

In [ ]:
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
import numpy as np
import copy
import requests

# Webscrape NFL Big Board

In [ ]:

# for loop getting required webpages
pages = range(1,5)

bb_frames = {}
for n in pages:        # NBA season to scrape
  page = n
  # URL to scrape, notice f string:
  bb_frames["bb_url" + str(n)] = f"https://www.drafttek.com/2024-NFL-Draft-Big-Board/Top-NFL-Draft-Prospects-2024-Page-{page}.asp"


In [ ]:
# contains each link used
bb_frames

In [ ]:
pl_bb_frames = {}

for key, url in bb_frames.items():
    response = requests.get(url)
    if response.status_code == 200:
        tables = pd.read_html(response.content)
        pl_bb_frames[key] = tables[1] if len(tables) > 1 else None
    else:
        print(f"Failed to fetch data from {url}")

In [ ]:
pl_bb_frames

In [ ]:
NFL_Draft_Player_DF = pd.concat(pl_bb_frames.values(), ignore_index=True)

# Use the first row as column headers
if not NFL_Draft_Player_DF.empty:
    NFL_Draft_Player_DF.columns = NFL_Draft_Player_DF.iloc[0]
    NFL_Draft_Player_DF = NFL_Draft_Player_DF[1:]  # Exclude the first row after setting as column headers


# Drop specified columns
columns_to_drop = ['CNG', 'Ht', 'Wt', 'Elig', 'TRAK']
NFL_Draft_Player_DF = NFL_Draft_Player_DF.drop(columns=columns_to_drop, errors='ignore')

# Rename 'P1' column to 'Position'
NFL_Draft_Player_DF = NFL_Draft_Player_DF.rename(columns={'P1': 'Position'})

In [ ]:
'''
NFL_Draft_Player_DF = pd.concat(pl_bb_frames)
#NFL_Draft_Player_DF = NBA_Player_DF.drop_duplicates()
NFL_Draft_Player_DF.reset_index(drop=True, inplace=True)
'''

In [ ]:

NFL_Draft_Player_DF['Rank'] = NFL_Draft_Player_DF['Rank'].astype(str)



In [ ]:
NFL_Draft_Player_DF

# Draft Order

In [ ]:
# read in the dataset
# Replace this with the published CSV link from your Google Sheets document
published_csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQm5tfir1M3cvUa_b69CHMJgvKZIRfk2yDG8GMhWqN90anthOojFm_u7nYWdgqhnBwvCFoxkS2QXf7P/pub?output=csv'

# Read the CSV file directly from the published link
NFL_Draft_Order_DF = pd.read_csv(published_csv_url)

NFL_Draft_Order_DF.rename(columns={'Unnamed: 1': 'Team'}, inplace=True)



In [ ]:
# Initialize the 'Round' column
NFL_Draft_Order_DF['Round'] = ''

# Iterate through the DataFrame and fill the 'Round' column
current_round = ''
for index, row in NFL_Draft_Order_DF.iterrows():
    pick = row['Pick']
    if pd.notnull(pick) and 'Round' in pick:
        current_round = pick
    NFL_Draft_Order_DF.at[index, 'Round'] = current_round

# Display the updated DataFrame
print(NFL_Draft_Order_DF)



In [ ]:
# Remove rows with null cells in any column and rows where 'Pick' contains 'Round'
NFL_Draft_Order_DF = NFL_Draft_Order_DF.dropna()  # Remove rows with null cells in any column
NFL_Draft_Order_DF = NFL_Draft_Order_DF[~NFL_Draft_Order_DF['Pick'].str.contains('Round')]  # Remove rows where 'Pick' contains 'Round'

# Reset index
NFL_Draft_Order_DF.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
print(NFL_Draft_Order_DF)


In [ ]:
'''
def remove_last_abbreviation(team_name):
    if isinstance(team_name, str):  # Check if it's a string
        # Split the string into parts separated by spaces
        parts = team_name.split()
        # Check if the last part contains only capital letters
        last_part = parts[-1]
        if last_part.isupper() and len(last_part) >= 2:
            # Remove the last part if it's all uppercase letters and at least 2 characters
            return ' '.join(parts[:-1])  # Join the parts except the last one
    return team_name  # Return unchanged if not a string or no abbreviation found


# Apply the function to the 'Team' column
#NFL_Draft_Order_DF['Team'] = NFL_Draft_Order_DF['Team'].apply(remove_last_abbreviation)

# Display the updated DataFrame
print(NFL_Draft_Order_DF)
'''
def remove_last_abbreviation(team_name):
    if isinstance(team_name, str):  # Check if it's a string
        # Split the string into parts separated by spaces
        parts = team_name.split()
        # Check if there are parts in the string
        if len(parts) > 1:
            # Check if the last part contains only capital letters
            last_part = parts[-1]
            if last_part.isupper() and len(last_part) >= 2:
                # Remove the last part if it's all uppercase letters and at least 2 characters
                return ' '.join(parts[:-1])  # Join the parts except the last one
    return team_name  # Return unchanged if not a string or no abbreviation found

# Apply the function to the 'Team' column
NFL_Draft_Order_DF['Team'] = NFL_Draft_Order_DF['Team'].apply(remove_last_abbreviation)

# Display the updated DataFrame
print(NFL_Draft_Order_DF)


In [ ]:
# Reorder columns to have 'Round' column as the first column
NFL_Draft_Order_DF = NFL_Draft_Order_DF[['Round'] + [col for col in NFL_Draft_Order_DF.columns if col != 'Round']]

# Display the updated DataFrame
print(NFL_Draft_Order_DF)


In [ ]:
NFL_Draft_Order_DF

# Team Needs

In [ ]:
# Replace this with the published CSV link from your Google Sheets document
published_csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQm5tfir1M3cvUa_b69CHMJgvKZIRfk2yDG8GMhWqN90anthOojFm_u7nYWdgqhnBwvCFoxkS2QXf7P/pub?gid=808119699&single=true&output=csv'

# Read the CSV file directly from the published link
NFL_Team_Needs_DF = pd.read_csv(published_csv_url)

In [ ]:
NFL_Team_Needs_DF

# Merged Data

In [ ]:
# merge draft order and needs dataframes
merged_df = pd.merge(NFL_Draft_Order_DF, NFL_Team_Needs_DF, on='Team', how='left')


In [ ]:
merged_df

In [ ]:
# create dataframe that will hold all infro including drafted player

final_df = pd.merge(merged_df, NFL_Draft_Player_DF, left_on=['Round', 'Pick'], right_on=['Rank', 'Prospect'], how='left')



In [ ]:
final_df

# Mock Draft

In [ ]:
# reset datafrme
updated_final_df = final_df
'''
# List of columns to clear
columns_to_clear = ["Rank", "Prospect", "College", "Position"]

# Clearing the data in specified columns
# Clearing the data in specified columns
updated_final_df[columns_to_clear] = updated_final_df[columns_to_clear].apply(lambda x: None)
'''

In [ ]:
import pandas as pd
from tabulate import tabulate
import copy

# Assuming your existing dataframes and initializations

def display_previous_selections(updated_final_df):
    selected_rows = updated_final_df.dropna(subset=['Rank'])
    if selected_rows.empty:
        print("No selections made yet.")
    else:
        columns_to_display = ['Round', 'Pick', 'Team', 'Prospect', 'College', 'Position']
        print(tabulate(selected_rows[columns_to_display], headers='keys', tablefmt='pretty', showindex=False))

def fill_missing_info(row, available_players, updated_final_df):
    valid_selection = False
    while not valid_selection:
        if pd.isnull(row['Rank']):
            print(f"Current Pick: {row['Round']} - Pick {row['Pick']} by {row['Team']}")

            team_needs = NFL_Team_Needs_DF[NFL_Team_Needs_DF['Team'] == row['Team']]
            if not team_needs.empty:
                print(f"Team Needs: {team_needs['Needs'].values[0]}\n")


            print("Top 10 Players Ranked:")
            for index, player in available_players.head(10).iterrows():
                player_positions = player['Position']
                print(f"{player['Rank']} - {player['Prospect']} (Positions: {player_positions})")
            print("\n")
            display_previous_selections(updated_final_df)  # Display selected players info as a table
            print("\n")
            print("Enter the Rank for the player you want to select.")
            print("Type 'exit' to finish the draft and show the picks made.")
            print("\n")
            chosen_player = input("Player Rank: ")
            print("\n")

            if chosen_player.lower() == 'exit':
                return 'exit', available_players

            player_exists = available_players[available_players['Rank'].str.lower() == chosen_player.lower()]
            if not player_exists.empty:
                row['Rank'] = player_exists.iloc[0]['Rank']
                row['Prospect'] = player_exists.iloc[0]['Prospect']
                row['College'] = player_exists.iloc[0]['College']
                row['Position'] = player_exists.iloc[0]['Position']

                selected_player = player_exists.iloc[0].copy()
                updated_final_df.loc[row.name, ['Rank', 'Prospect', 'College', 'Position']] = selected_player[['Rank', 'Prospect', 'College', 'Position']]

                available_players.drop(player_exists.index, inplace=True)
                available_players.reset_index(drop=True, inplace=True)

                valid_selection = True
            else:
                print("Invalid selection. Please choose a valid rank from the list.\n")

    return row, available_players

def initiate_mock_draft():
    global NFL_Draft_Player_DF, updated_final_df

    NFL_Draft_Player_DF = copy.deepcopy(copy_NFL_Draft_Player_DF)
    updated_final_df = copy.deepcopy(copy_updated_final_df)

    available_players = original_available_players.copy()

    updated_final_df, available_players = updated_final_df.apply(
        fill_missing_info, axis=1, available_players=available_players, updated_final_df=updated_final_df
    )

    while True:
        if updated_final_df == 'exit':
            print("Mock draft stopped.")
            break

        choice = input("Do you want to save the mock draft? (yes/no): ")
        if choice.lower() == 'yes':
            updated_final_df.to_csv('saved_draft.csv', index=False)
            print("Mock draft saved.")
            break
        elif choice.lower() == 'no':
            print("Starting a new mock draft.")
            initiate_mock_draft()
            break
        else:
            print("Please enter 'yes' or 'no'.")

initiate_mock_draft()


In [ ]:
updated_final_df